<a href="https://colab.research.google.com/github/esnue/ThesisAllocationSystem/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>